<a href="https://colab.research.google.com/github/deepthipyata/Speech-Emotion-Recognition/blob/main/ser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
pip install librosa soundfile numpy sklearn

In [ ]:
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libasound2-dev is already the newest version (1.1.3-5ubuntu0.5).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 34 not upgraded.
Need to get 184 kB of archives.
After this operation, 891 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudiocpp0 amd64 19.6.0-1 [15.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 portaudio19-dev amd64 19.6.0-1 [104 kB]
Fetched 184 kB in 1s (345 kB/s)
Selecting pr

In [ ]:
!pip install pyaudio

  Using cached https://files.pythonhosted.org/packages/ab/42/b4f04721c5c5bfc196ce156b3c768998ef8c0ae3654ed29ea5020c749a6b/PyAudio-0.2.11.tar.gz
  Created wheel for pyaudio: filename=PyAudio-0.2.11-cp37-cp37m-linux_x86_64.whl size=52578 sha256=88cf9a015a3a254c28e9ba9c2ef40fc3bdfb617efe50dec854ccafe904e36fba
  Stored in directory: /root/.cache/pip/wheels/f4/a8/a4/292214166c2917890f85b2f72a8e5f13e1ffa527c4200dcede
Successfully built pyaudio


librosa is a Python library for analyzing audio and music. It has a flatter package layout, standardizes interfaces and names, backwards compatibility, modular functions, and readable code. 

In [15]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

### mfcc: Mel Frequency Cepstral Coefficient, represents the short-term power spectrum of a sound
### chroma: Pertains to the 12 different pitch classes
### mel: Mel Spectrogram Frequency

In [37]:
#Feature_Extraction
#extracting mfcc, chroma, mel from soundfiles all these three are boolean parameters forthree features

def feature_extraction(audiofile,mfcc,chroma,mel):
  with soundfile.SoundFile(audiofile) as audio_file:
    read_audio=audio_file.read(dtype="float32")
    samplerate=audio_file.samplerate
    if chroma:  #if chroma is true apply short time fourier transform
      stft = np.abs(librosa.stft(read_audio))
    result_audio=np.array([])
    if mfcc:
      mfccf=np.mean(librosa.feature.mfcc(y=read_audio,sr=samplerate,
                                         n_mfcc=40).T, axis=0)
      result_audio=np.hstack((result_audio,mfccf))
    if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=samplerate).T,axis=0)
            result_audio=np.hstack((result_audio, chroma))
    if mel:
            mel=np.mean(librosa.feature.melspectrogram(read_audio, sr=samplerate).T,axis=0)
            result_audio=np.hstack((result_audio, mel))
    return result_audio


In [49]:
#Dictionary for various emotions
emotions={
    '01':'neutral',
    '02':'calm',
    '03':'happy',
    '04':'sad',
    '05':'angry',
    '06':'fearful',
    '07':'disgust',
    '08':'surprised',
    
}
#in dataset we have clm,happy, fearful, disgust

emotions_DS=['happy','fearful','disgust','calm']





In [50]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [51]:
#path="\\content\\drive\\MyDrive\\PythonDataSets\\SER\\Actor_*\\*.wav"
path="/content/drive/MyDrive/PythonDataSets/SER/Actor_*/*.wav"

In [53]:
#DataFlair - Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob(path):
        fname=os.path.basename(file)
        #print(fname)
        #print(fname.split("-")[2])
        emotion=emotions[fname.split("-")[2]]
        #print(emotion)
        #if emotion not in emotions_DS:
            #continue
        feature=feature_extraction(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [54]:
#Split the dataset
x_train,x_test,y_train,y_test= load_data(test_size=0.25)

In [55]:
print((x_train.shape[0],x_test.shape[0]))

(1080, 360)


In [56]:
print(f'Features extracted: {x_train.shape[1]}')
x_train.shape

Features extracted: 180


(1080, 180)

In [58]:
#DataFlair - Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)